In [5]:
from simulator import SimulatorHandler

In [6]:
class FCFSScheduler:
    def __init__(self, simulator: SimulatorHandler) -> None:
        self.simulator = simulator

    def __str__(self) -> str:
        return "FCFS"

    def schedule(self) -> None:
        """  First Come First Served policy """
        assert self.simulator.is_running

        for job in self.simulator.queue:
            available = self.simulator.platform.get_not_allocated_hosts()

            if job.res <= len(available):
                # Schedule if the job can start now.
                allocation = [h.id for h in available[:job.res]]
                self.simulator.allocate(job.id, allocation)
            else:
                # Otherwise, wait for resources.
                break

In [7]:
class EASYScheduler(FCFSScheduler):
    def __str__(self) -> str:
        return "EASY"

    def schedule(self) -> None:
        super().schedule() # Schedule with FCFS

        # Apply the backfilling mechanism
        if len(self.simulator.queue) >= 2:
            self.backfill()

    def backfill(self) -> None:
        assert len(self.simulator.queue) >= 2

        # The priority job is the first job in the queue.
        p_job = self.simulator.queue[0]

        # The remaining jobs can be scheduled if they do not delay p_job.
        backfilling_queue = self.simulator.queue[1:]

        # Get the next expected releases
        next_releases = sorted(self.simulator.agenda, key=lambda a: a.release_time)

        # Get the last required host for p_job.
        last_host = next_releases[p_job.res - 1]

        # The last host release time will be the p_job expected start time.
        p_start_t = last_host.release_time

        # Find candidates and reserve resources for p_job.
        candidates = [r.host.id for r in next_releases if r.release_time <= p_start_t]

        # Try to maximize the number of hosts available for the remaining queue.
        reservation = candidates[-p_job.res:]

        # Let's try to start some jobs earlier.
        for job in backfilling_queue:
            available = self.simulator.platform.get_not_allocated_hosts()  # Hosts
            not_reserved = [h for h in available if h.id not in reservation]

            if job.res <= len(not_reserved):
                # Schedule job on not reserved hosts.
                allocation = [h.id for h in not_reserved[:job.res]]
                self.simulator.allocate(job.id, allocation)
            elif job.walltime and job.walltime <= p_start_t and job.res <= len(available):
                # Schedule job on reserved hosts without delaying p_job.
                allocation = [h.id for h in available[:job.res]]
                self.simulator.allocate(job.id, allocation)

In [12]:
def run_simulation(scheduler):
    simulator = SimulatorHandler()
    scheduler = scheduler(simulator)

    # 1) Instantiate monitors to collect simulation statistics
    jobs_mon = JobMonitor(simulator)
    sim_mon = SimulationMonitor(simulator)

    # 2) Start simulation
    simulator.start(platform="platforms/hosts_2.xml",
                    workload="workloads/comp_4.json",
                    verbosity="information")

    # 3) Schedule all jobs
    while simulator.is_running:
        scheduler.schedule()
        simulator.proceed_time()  # proceed directly to the next event.
    simulator.close()

    # 4) Return/Dump statistics
    return jobs_mon, sim_mon

jobs_f, sim_f = run_simulation(FCFSScheduler)
jobs_e, sim_e = run_simulation(EASYScheduler)

ImportError: (HINT: you need to install Batsim. Check the setup instructions here: https://batsim.readthedocs.io/en/latest/.). Run "batsim --version" to make sure it is working.

In [11]:
from monitors import JobMonitor, SimulationMonitor